<a href="https://colab.research.google.com/github/catherinebirney/tiem-training/blob/main/Notebooks/stewi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Standardized Emission and Waste Inventories (StEWI)

StEWI is a collection of Python modules that provide processed USEPA facility-based emission and waste generation inventory data in standard tabular formats. The standard outputs may be further aggregated or filtered based on given criteria, and can be combined based on common facility and flows across the inventories.

In [ ]:
# Clone the repository from GitHub into the local colab environment
!git clone https://github.com/USEPA/standardizedinventories.git

In [ ]:
# Install the locally cloned repository to have access to the root files
%cd standardizedinventories
!pip install -e .

In [ ]:
import stewi
import pandas as pd
import numpy as np

## Accessing Data
Data in StEWI can be accessed in a number of formats:
- [Flow](https://github.com/USEPA/standardizedinventories/blob/master/format%20specs/Flow.md)
- [Facility](https://github.com/USEPA/standardizedinventories/blob/master/format%20specs/Facility.md)
- [FlowByFacility](https://github.com/USEPA/standardizedinventories/blob/master/format%20specs/FlowByFacility.md)
- [FlowByProcess](https://github.com/USEPA/standardizedinventories/blob/master/format%20specs/FlowByProcess.md)

Attempting to retreive data for the first time (e.g., via `stewi.getInventory()` or `stewi.getInventoryFacilities()`) will cause StEWI to run the appropriate modules to access the public data directly from their source. Alternatively, use `download_if_missing=True` to obtain the StEWI processed data that has been validated and published to the [EPA Data Commons](https://dmap-data-commons-ord.s3.amazonaws.com/index.html?prefix=#stewi/). StEWI will always use data stored locally first, before attempting to download or generate the data.

In [ ]:
# Review the available data in StEWI
stewi.getAllInventoriesandYears()

In [ ]:
# Download and review the 2017 NEI
NEI = stewi.getInventory('NEI', 2017, download_if_missing=True)
print(NEI)
print(f'Number of unique flows: {len(NEI.FlowName.unique())}')
print(f'Number of unique facilities: {len(NEI.FacilityID.unique())}')

NEI and GHGRP are also available as `FlowByProcess` format, which provides the data at sub-facility level. For the NEI, this means emissions are reported by facility by [Source Classification Code (SCC)](https://sor-scc-api.epa.gov/sccwebservices/sccsearch/) while the GHGRP is aggregated by [GHGRP Subpart](https://www.epa.gov/ghgreporting/resources-subpart-ghg-reporting).

In [ ]:
# NEI and GHGRP are also available as FlowByProcess
GHGRP = stewi.getInventory('GHGRP', year=2021, stewiformat='flowbyprocess', download_if_missing=True)
print(GHGRP)
print(f'Available GHGRP Subparts: {GHGRP.Process.unique()}')

In [ ]:
# Review the totals by Flow
GHGRP.pivot_table(index=['FlowName', 'Unit'], values='FlowAmount', columns='Process',
                  aggfunc='sum', margins=True, margins_name='Total', fill_value=0)

The facilities file contains detailed information on every facility reporting to the program. In the case of eGRID, this includes additional details on facility resource mix.

In [ ]:
# Review facilities in eGRID
egrid = stewi.getInventoryFacilities('eGRID', 2018, download_if_missing=True)
print(egrid.columns)
print(egrid)

## Challenge Questions
1. What state reports the most facilities for a dataset? Does it change over time?
2. What are the top 3 GHGRP Subparts with the most Methane emissions?
3. How many pollutants (emissions to air) are reported in eGRID?


## Extreme TIEM Challenge
What is the total characterized impacts (i.e., Global Warming Potential, Smog Formation Potential, etc.) of all reported facilities from a dataset?
  - Hint 1: Use [apply_flow_mapping](https://github.com/USEPA/esupy/blob/main/esupy/mapping.py#L11-L12) from `esupy.mapping` to convert the flows from a data source to the FEDEFL.
  - Hint 2: Use [apply_lcia_method](https://github.com/USEPA/LCIAformatter/blob/master/lciafmt/__init__.py#L268) from `lciafmt` to apply your desired LCIA method.

# stewicombo
The `stewicombo` module of [StEWI](https://github.com/USEPA/standardizedinventories) combines inventory data from within and across selected inventories by matching facilities in the [Facility Registry Service](https://www.epa.gov/frs) and chemical flows using the [Substance Registry Service](https://sor.epa.gov/sor_internet/registry/substreg/LandingPage.do).
If the `remove_overlap` parameter is set to True (default), `stewicombo` combines records using the following default logic:
- Records that share a common compartment, SRS ID and FRS ID _within_ an inventory are summed.
- Records that share a common compartment, SRS ID and FRS ID _across_ an inventory are assessed by compartment preference (see `INVENTORY_PREFERENCE_BY_COMPARTMENT`).
- Additional steps are taken to avoid overlap of:
    - nutrient flow releases to water between the TRI and DMR
    - particulate matter releases to air reflecting PM < 10 and PM < 2.5 in the NEI
    - [Volatile Organic Compound (VOC)](https://github.com/USEPA/standardizedinventories/blob/master/stewicombo/data/VOC_SRS_IDs.csv) releases to air for individually reported VOCs and grouped VOCs

In [ ]:
import stewicombo
import facilitymatcher
import chemicalmatcher

In [ ]:
# Download and review the dataset of facility matches
fac_matches = facilitymatcher.get_matches_for_inventories()

In [ ]:
print(f'Datasets found: {fac_matches.Source.unique()}')
fac_matches

In [ ]:
# Combine select datasets
inv_dict = {"NEI":"2017", "TRI":"2017"}
df = stewicombo.combineFullInventories(inv_dict, filter_for_LCI=True,
                                        remove_overlap=True,
                                        compartments=['air'],
                                        download_if_missing=True)

In [ ]:
df